## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-18-23-50-54 +0000,nypost,Mount Sinai hospital shoots down claims that p...,https://nypost.com/2026/01/18/us-news/mount-si...
1,2026-01-18-23-46-01 +0000,startribune,"Maye throws 3 TD passes, Stroud has 4 INTs as ...",https://www.startribune.com/maye-throws-3-td-p...
2,2026-01-18-23-46-00 +0000,wsj,Precious Metals Climb After Trump Threatens Eu...,https://www.wsj.com/finance/commodities-future...
3,2026-01-18-23-45-52 +0000,bbc,'An ambassador for African football' - Mane is...,https://www.bbc.com/sport/football/articles/cl...
4,2026-01-18-23-37-14 +0000,bbc,At least 21 killed in Spain after crash involv...,https://www.bbc.com/news/articles/cedw6ylpynyo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2392/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
28,trump,44
32,greenland,19
170,ice,12
76,iran,8
31,tariffs,7


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
150,2026-01-18-08-00-32 +0000,wapo,Melting ice may raise Greenland’s value. Trump...,https://www.washingtonpost.com/climate-environ...,91
83,2026-01-18-17-37-12 +0000,nypost,GOP lawmakers irked at Trump’s new planned ta...,https://nypost.com/2026/01/18/us-news/gopers-i...,87
73,2026-01-18-18-20-27 +0000,nypost,EU threatens trade ‘bazooka’ against US after ...,https://nypost.com/2026/01/18/world-news/europ...,86
93,2026-01-18-16-09-10 +0000,cbc,Canada 'concerned' about Trump's Greenland tar...,https://www.cbc.ca/news/politics/canada-greenl...,84
158,2026-01-18-06-01-39 +0000,nyt,Trump Announces New Tariffs on Some European C...,https://www.nytimes.com/2026/01/17/us/politics...,84


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
150,91,2026-01-18-08-00-32 +0000,wapo,Melting ice may raise Greenland’s value. Trump...,https://www.washingtonpost.com/climate-environ...
8,35,2026-01-18-23-24-23 +0000,nypost,Pregnant mother of two ID’d among six women ki...,https://nypost.com/2026/01/18/world-news/pregn...
12,31,2026-01-18-23-14-31 +0000,cbc,At least 20 believed dead in high-speed train ...,https://www.cbc.ca/news/world/spain-train-cras...
81,30,2026-01-18-17-50-47 +0000,nypost,Minneapolis Mayor Jacob Frey rips ICE’s immigr...,https://nypost.com/2026/01/18/us-news/minneapo...
102,29,2026-01-18-14-56-16 +0000,nypost,President Trump puts $1B price tag on permanen...,https://nypost.com/2026/01/18/us-news/presiden...
112,28,2026-01-18-13-00-00 +0000,startribune,Wondering what to do indoors with kids on a co...,https://www.startribune.com/best-indoor-playgr...
67,27,2026-01-18-19-15-11 +0000,nypost,Stunned woman wakes up to find 8-foot python o...,https://nypost.com/2026/01/18/world-news/stunn...
83,24,2026-01-18-17-37-12 +0000,nypost,GOP lawmakers irked at Trump’s new planned ta...,https://nypost.com/2026/01/18/us-news/gopers-i...
34,23,2026-01-18-21-42-16 +0000,nyt,Noem Denies Use of Chemical Agents in Minnesot...,https://www.nytimes.com/2026/01/18/us/politics...
126,22,2026-01-18-10-34-00 +0000,wsj,Answers to frequently asked questions about Th...,https://www.wsj.com/economy/economic-forecasti...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
